In [25]:
from langchain.docstore.document import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
from langchain.indexes import VectorstoreIndexCreator

In [27]:

load_dotenv()

loader = CSVLoader(file_path="../../documents/example.csv")


In [ ]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
# Query the vector store
query = "What is the seventh item in the OWASP Top 10?"
result = index.query(query)
print(result)

In [30]:
docs = loader.load()

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [33]:
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [34]:
# test retrieval
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain


llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [39]:

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "What is the seventh item in the OWASP Top 10 in 2017? and what is it in 2021"})
response

In [41]:
vector.save_local("../vector_store/owasp_faiss")